In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [17]:
import requests
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate


def weather_search(city):
    response = requests.get(
        url=f"https://api.weatherapi.com/v1/current.json?key=6462b585c894452484d224329250102&q={city}"
    )

    return str(response.json())


weather_details_prompt = PromptTemplate.from_template(
    "Summarize the weather condition: {city_weather}"
)
weather_details_chain = LLMChain(
    llm=llm, prompt=weather_details_prompt, output_key="weather_summary"
)

time_spend_suggestions = PromptTemplate.from_template(
    """Given this weather condition, suggest activities:\n\nWeather: {weather_summary}"""
)
time_spend_chain = LLMChain(
    llm=llm, prompt=time_spend_suggestions, output_key="activity_suggestions"
)

sequential_chain = SequentialChain(
    chains=[weather_details_chain, time_spend_chain],
    input_variables=["city_weather"],
    output_variables=["activity_suggestions"],
    verbose=True,
)

In [18]:
weather_info = weather_search("Chandigarh")
weather_info

"{'location': {'name': 'Chandigarh', 'region': 'Chandigarh', 'country': 'India', 'lat': 30.7372, 'lon': 76.7872, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1738451934, 'localtime': '2025-02-02 04:48'}, 'current': {'last_updated_epoch': 1738451700, 'last_updated': '2025-02-02 04:45', 'temp_c': 12.4, 'temp_f': 54.2, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 31, 'wind_dir': 'NNE', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 40, 'cloud': 3, 'feelslike_c': 11.9, 'feelslike_f': 53.3, 'windchill_c': 11.9, 'windchill_f': 53.3, 'heatindex_c': 12.4, 'heatindex_f': 54.2, 'dewpoint_c': -0.7, 'dewpoint_f': 30.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.9, 'gust_kph': 15.9}}"

In [19]:
response = sequential_chain.invoke(
    {"city_weather": weather_info}
)  # ✅ Pass the fetched weather data
print(response)




> Entering new SequentialChain chain...

> Finished chain.
The clear and mild nighttime weather in Chandigarh presents several enjoyable activities. Here are some suggestions:

1. **Stargazing**: With clear skies and good visibility, this is a perfect opportunity to look up at the stars. Find a quiet spot away from city lights, bring a blanket, and enjoy the night sky.

2. **Evening Walks**: Take a refreshing stroll in one of Chandigarh's parks or gardens like Sukhna Lake or Rose Garden. The cool breeze and clear weather make for a pleasant walking experience.

3. **Outdoor Dining**: Consider visiting an outdoor café or restaurant. Enjoy a meal or coffee while taking advantage of the nice weather.

4. **Picnic Night**: Pack a light dinner and head to a park for a nighttime picnic. The calm weather creates a cozy atmosphere for eating under the stars.

5. **Photography**: If you're into photography, this weather is great for capturing nighttime shots. The visibility will help you take

In [21]:
weather_details_chain

LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['city_weather'], input_types={}, partial_variables={}, template='Summarize the weather condition: {city_weather}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f50fd7a28a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f50fd7bc680>, root_client=<openai.OpenAI object at 0x7f51144a6090>, root_async_client=<openai.AsyncOpenAI object at 0x7f50fd7a2900>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_key='weather_summary', output_parser=StrOutputParser(), llm_kwargs={})